In [6]:
import numpy as np
import cv2
import pickle

In [7]:
ESCAPE_KEY = 27
SPACE_KEY = 32

In [8]:
# Get camera feed
vid = cv2.VideoCapture(0) 
images = []
space_down = False

while(True): 
    _, frame = vid.read() 
  
    # Display the resulting frame 
    cv2.imshow('frame', frame) 
    
    # Save frame if spacebar, stop if escape
    key = cv2.waitKey(1)
    if key == ESCAPE_KEY: 
        break
    if (key == SPACE_KEY) and (not space_down): 
        space_down = True
        images.append(frame)
    if (key != SPACE_KEY and space_down):
        space_down = False
  
vid.release() 
cv2.destroyAllWindows() 

In [9]:
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

objp = np.zeros((10*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:10].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
for img in images: 
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7,10), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)
        # Draw and display the corners
        cv2.drawChessboardCorners(img, (7,10), corners2, ret)
        cv2.imshow('img', img)
        cv2.waitKey(500)
cv2.destroyAllWindows()

In [10]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
calib_params = {'mtx':mtx, 'dist':dist}
with open('calibration_parameters.pickle', 'wb') as file:
    pickle.dump(calib_params, file)